#ROOT pipeline demonstration

In [ ]:
import tensorflow as tf
tf.VERSION

In [ ]:
%%bash
mkdir Ops/build || rm -rf Ops/build/*
cd Ops/build
cmake -DCMAKE_INSTALL_PREFIX=$PWD/release ..
make
make install

In [ ]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(),'Ops','build','release'))

In [ ]:
import rtf
rtf.root_reader